In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
import json
with open('../input/file-paths-dataset/file_paths.json', 'r') as fp:
    json_data = json.load(fp)

In [ ]:
choosen_indices = ['4', '5']

In [ ]:
list_1 = json_data[choosen_indices[0]]
list_2 = json_data[choosen_indices[1]]

In [ ]:
list_1 = list_1[:20]

In [ ]:
!conda install -y gdown 
import gdown

In [ ]:
# https://drive.google.com/file/d/18iv0C9LuRXLj1WdubzpWIDdDTzZPdah4/view?usp=sharing
# https://drive.google.com/file/d/1aFxYpCOjoM03KIu7OtCUxkaN1s6z1qd0/view?usp=sharing
url = 'https://drive.google.com/uc?id=1aFxYpCOjoM03KIu7OtCUxkaN1s6z1qd0' 
output = 'playlist.tar.gz'
gdown.download(url, output)

In [ ]:
!tar -xzf ./playlist.tar.gz

In [ ]:
!conda install -c conda-forge ffmpeg libsndfile -y

In [ ]:
!pip install spleeter

In [ ]:
import tensorflow as tf

In [ ]:
!pip install --upgrade tensorflow-hub

In [ ]:
import tensorflow_hub as hub
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
!pip install scipy

In [ ]:
from scipy.io import wavfile

In [ ]:
from scipy.io import wavfile
import scipy.signal

def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

# sample_rate, wav_data = wavfile.read('./output/Anjaana_Anjaani/accompaniment.wav', 'rb')
# sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)

In [ ]:
!rm ./playlist.tar.gz

In [ ]:
import os
import subprocess
from collections import defaultdict
import torchaudio

index = 0

hash_map_music = defaultdict(lambda: defaultdict(list))

index = 0
break_out = False
for dirname, _, filenames in os.walk('./playlist'):
    for filename in filenames:
        if ''.join(filename[-3:]) == 'mp3':
            file_loc = dirname + '/' + filename
            file_loc_truncated = file_loc.replace(' ', '')
            file_key = 'music_for_you/' + "".join(file_loc_truncated[2:])
            if file_key in list_1:
                print(dirname, filename, index)
                subprocess.call(['spleeter', 'separate', '-p', 'spleeter:2stems', '-o', 'output', file_loc])
                directory_name = ''.join(filename[:-4])

                sample_rate, wav_data = wavfile.read('./output/' + directory_name + '/accompaniment.wav', 'rb')
                sample_rate_beats, wav_data_beats = ensure_sample_rate(sample_rate, wav_data)
                wav_data_beats = wav_data_beats.ravel()
                scores_beats, embeddings_beats, spectrogram_beats = yamnet_model(wav_data_beats)

                sample_rate, wav_data = wavfile.read('./output/' + directory_name + '/vocals.wav', 'rb')
                sample_rate_vocals, wav_data_vocals = ensure_sample_rate(sample_rate, wav_data)
                wav_data_vocals = wav_data_vocals.ravel()
                scores_vocals, embeddings_vocals, spectrogram_vocals = yamnet_model(wav_data_vocals)

                wav_data, sample_rate = torchaudio.load(file_loc)
                sample_rate_song, wav_data_song = ensure_sample_rate(sample_rate, wav_data)
                wav_data_song = wav_data_song.ravel()
                scores_song, embeddings_song, spectrogram_song = yamnet_model(wav_data_song)

                file_loc_truncated = file_loc.replace(' ', '')
                file_key = 'music_for_you/' + "".join(file_loc_truncated[2:])
                hash_map_music[file_key]['beats'] = embeddings_beats.numpy()
                hash_map_music[file_key]['vocals'] = embeddings_vocals.numpy()
                hash_map_music[file_key]['song'] = embeddings_song.numpy()
                index += 1         

In [ ]:
new_hash_music_dictionary = defaultdict(lambda: defaultdict(list))
for key, val in hash_map_music.items():
	for key_inner, val_inner in val.items():
		new_hash_music_dictionary[key][key_inner] = val_inner.tolist()

with open('hash_map_4.json', 'w') as fp:
    json.dump(new_hash_music_dictionary, fp, indent=4)

In [ ]:
# !rm ./playlist.tar.gz

In [ ]:
import os
import subprocess
from collections import defaultdict
import torchaudio

index = 0

hash_map_music = defaultdict(lambda: defaultdict(list))

index = 0
break_out = False
for dirname, _, filenames in os.walk('./playlist'):
    for filename in filenames:
        if ''.join(filename[-3:]) == 'mp3':
            file_loc = dirname + '/' + filename
            file_loc_truncated = file_loc.replace(' ', '')
            file_key = 'music_for_you/' + "".join(file_loc_truncated[2:])
            if file_key in list_2:
                print(dirname, filename, index)
                subprocess.call(['spleeter', 'separate', '-p', 'spleeter:2stems', '-o', 'output', file_loc])
                directory_name = ''.join(filename[:-4])

                sample_rate, wav_data = wavfile.read('./output/' + directory_name + '/accompaniment.wav', 'rb')
                sample_rate_beats, wav_data_beats = ensure_sample_rate(sample_rate, wav_data)
                wav_data_beats = wav_data_beats.ravel()
                scores_beats, embeddings_beats, spectrogram_beats = yamnet_model(wav_data_beats)

                sample_rate, wav_data = wavfile.read('./output/' + directory_name + '/vocals.wav', 'rb')
                sample_rate_vocals, wav_data_vocals = ensure_sample_rate(sample_rate, wav_data)
                wav_data_vocals = wav_data_vocals.ravel()
                scores_vocals, embeddings_vocals, spectrogram_vocals = yamnet_model(wav_data_vocals)

                wav_data, sample_rate = torchaudio.load(file_loc)
                sample_rate_song, wav_data_song = ensure_sample_rate(sample_rate, wav_data)
                wav_data_song = wav_data_song.ravel()
                scores_song, embeddings_song, spectrogram_song = yamnet_model(wav_data_song)

                file_loc_truncated = file_loc.replace(' ', '')
                file_key = 'music_for_you/' + "".join(file_loc_truncated[2:])
                hash_map_music[file_key]['beats'] = embeddings_beats.numpy()
                hash_map_music[file_key]['vocals'] = embeddings_vocals.numpy()
                hash_map_music[file_key]['song'] = embeddings_song.numpy()
                index += 1         

In [ ]:
new_hash_music_dictionary = defaultdict(lambda: defaultdict(list))
for key, val in hash_map_music.items():
	for key_inner, val_inner in val.items():
		new_hash_music_dictionary[key][key_inner] = val_inner.tolist()

with open('hash_map_1.json', 'w') as fp:
    json.dump(new_hash_music_dictionary, fp, indent=4)